# Search Term Ranker

## Search Term Params

In [ ]:
file = r"C:\Users\Tin Ha\Downloads\Tin 7D ST Report (13).xlsx" # Input File
new_file = "test2.xlsx" # Output File Name

min_cvr = .1         
min_acos = .001      
max_acos = .3        
min_clicks = 14      
min_orders = 2

BANNED_CHARS_LIST = [
    "/",
    "-"
]
BANNED_CHARS = "|".join(BANNED_CHARS_LIST)


### Import 7 Day ST Report

In [ ]:
import pandas as pd
import datetime as dt
import os

from config import create_ods_clickhouse_engine

def get_today():
    date = dt.date.today()
    date = date.strftime("%Y-%m-%d")
    return date



def get_productads(date, sales_channel_id=1111, market="US"):
    params = {
        "date":date,
        "sales_channel_id":sales_channel_id,
        "market":market
    }
    query = """
        SELECT p.asin as asin, p.sku as sku, p.adgroupid as adgroupid, a.name as adgroupname,  c.name as campaignname
        FROM ods.amz_ads_sp_productads p
        LEFT JOIN ods.amz_ads_sp_adgroups a
        ON a.adgroupid = p.adgroupid
        LEFT JOIN ods.amz_ads_sp_campaigns c
        ON c.campaignid = p.campaignid
        WHERE a.sales_channel_id = %(sales_channel_id)s
        AND p.sales_channel_id = %(sales_channel_id)s
        AND c.sales_channel_id = %(sales_channel_id)s
        AND a.date = %(date)s
        AND c.date = %(date)s
        AND p.date = %(date)s
        AND a.market = %(market)s
        AND c.market = %(market)s
        AND p.market = %(market)s
        AND p.state != 'archived'
        and a.state != 'archived'
        and c.state != 'archived'
        and p.asin IS NOT NULL         
        AND p.campaignid in (
            SELECT campaignid 
            FROM ods.amz_ads_sp_campaigns
            WHERE date = %(date)s
            AND state != 'archived'
            AND market = %(market)s
            AND sales_channel_id = %(sales_channel_id)s
        )
"""
    df = pd.read_sql(query,create_ods_clickhouse_engine(),params=params)
    return df


In [ ]:
today = get_today()
df = get_productads(today)
df2 = pd.read_excel(file, parse_dates=["Date"])

In [ ]:
BANNED_CHARS_LIST = [
    "/",
    "-"
]
BANNED_CHARS = "|".join(BANNED_CHARS_LIST)

df3 = df2[df2["Campaign Name"].str.contains("_iso_",case=False)]
df3 = df3[((df3["Match Type"]=="-") & (df3["Targeting"]=="*")) | (df3["Match Type"]=="BROAD")]
df3 = df3[~df3["Customer Search Term"].str.contains(BANNED_CHARS)]
df3 = df3.rename(columns={"Ad Group Name":"adgroupname","7 Day Advertised SKU Sales ":"Sales","7 Day Advertised SKU Units (#)":"Orders"})

In [ ]:
df_final = df3.merge(df,how="inner")
df_final = df_final[["Date","asin","Customer Search Term","Impressions","Clicks","Spend","Sales", "Orders"]]
df_final

In [ ]:
df_final.drop(columns=["Date"],inplace=True)
df_pivot = df_final.groupby(["asin","Customer Search Term"]).sum()
df_pivot["CPC"] = df_pivot.Spend/df_pivot.Clicks
df_pivot["ACOS"] = df_pivot.Spend/df_pivot.Sales
df_pivot["CVR"] = df_pivot.Orders/df_pivot.Clicks


In [ ]:
df_pivot.reset_index(drop=False,inplace=True)

In [ ]:
df_negative = df_pivot[((df_pivot["ACOS"] == float("inf"))|(df_pivot["ACOS"] >= 1)) & (df_pivot["Clicks"]>=min_clicks)]
df_negative.sort_values(by=["Spend"],ascending=False)


In [ ]:
df_good = df_pivot[(df_pivot["Clicks"] >= min_clicks) & (df_pivot.ACOS < max_acos) & (df_pivot.Orders >= min_orders) & (df_pivot.ACOS >= min_acos) & (df_pivot.CVR >= min_cvr)].sort_values(["Sales"],ascending=False)





In [ ]:


with pd.ExcelWriter(new_file) as writer:
    df_good.to_excel(writer,index=False,sheet_name="Good")
    df_negative.to_excel(writer,index=False,sheet_name="Negatives")
    df_pivot.to_excel(writer,index=False,sheet_name="Raw Data")

os.system(f"START EXCEL.EXE {new_file}")